In [1]:
import tensorflow as tf  # TensorFlow kütüphanesini içe aktarıyoruz.
from keras.preprocessing.image import ImageDataGenerator  # Görüntü verilerini artırmak ve işlemek için kullanılan sınıf.
from tensorflow.keras.models import Sequential  # Model oluşturmak için kullanılan Sequential sınıfı.
from tensorflow.keras import datasets, layers, models  # TensorFlow'dan gerekli modüller.
import matplotlib.pyplot as plt  # Grafikler oluşturmak için kullanılan kütüphane.
import os  # İşletim sistemi ile ilgili işlemler için kullanılan kütüphane.
import numpy as np  # Sayısal işlemler ve veri yönetimi için kullanılan kütüphane.
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout  # CNN katmanları için gerekli modüller.
from keras.optimizers import Adam  # Model optimizasyonu için Adam optimizasyon algoritması.
from keras.losses import mean_squared_error  # Kayıp fonksiyonu olarak "mean_squared_error" kullanımı.
from keras.metrics import mean_squared_error  # Değerlendirme metriği olarak "mean_squared_error" kullanımı.
import cv2  # Görüntü işleme için kullanılan OpenCV kütüphanesi.
import matplotlib.pyplot as plt  # Görselleştirme için matplotlib kütüphanesi tekrar çağrılmış (tekrarlanan çağrı gereksizdir).
from PIL import Image  # Görüntü işleme için Python Imaging Library (Pillow).
from keras.applications.inception_v3 import preprocess_input  # InceptionV3 modeli için gerekli ön işleme fonksiyonu.
from keras.preprocessing import image  # Görüntü işlemede yardımcı olan fonksiyonlar.
from tensorflow.keras.preprocessing import image  # TensorFlow versiyonu da kullanılmış (aynı işlevsellik için iki farklı çağrı yapılmış, gereksiz tekrar).
from tensorflow.keras.callbacks import EarlyStopping  # Eğitim sırasında erken durdurma için kullanılan sınıf.
from tensorflow.keras.models import load_model  # Kaydedilmiş modelleri yüklemek için kullanılan fonksiyon.

In [3]:
# Eğitim veri artırma (data augmentation) işlemi
train_datagen = ImageDataGenerator(rescale=1./255, # Görüntü piksel değerlerini 0-1 aralığına ölçeklendirir.
                                  shear_range = 0.2, # Görüntülere kaydırma (shear) işlemi uygular.
                                  zoom_range = 0.2, # Görüntülere rastgele yakınlaştırma uygular.
                                  horizontal_flip = True) # Görüntüleri yatay olarak rastgele çevirir.

# Eğitim verilerini belirtilen klasörden okuyarak ve veri artırma işlemleri uygulayar
training_set = train_datagen.flow_from_directory(r"C:\Users\kalybeai\Desktop\Sunstorm Analysis System\masked_continuum\train", # Eğitim verilerinin bulunduğu klasör.
                                                target_size=(256,256), # Görüntülerin yeniden boyutlandırılacağı hedef boyutlar (256x256 piksel).
                                                batch_size = 32, # Her bir eğitim turunda (epoch) modele aynı anda verilecek 32 görüntülük grup.
                                                class_mode ='categorical') # Sınıflar arasında kategorik sınıflandırma yapılacağını belirtir.

Found 10538 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)  # Test verilerindeki piksel değerlerini 0-1 aralığına ölçeklendirir.

# Test verilerini belirtilen klasörden alıp modele değerlendirme için uygun hale getiriyor.
test_set = test_datagen.flow_from_directory(
    r"C:\\Users\\kalybeai\\Desktop\\Sunstorm Analysis System\\masked_continuum\\test",  # Test verilerinin bulunduğu klasör.
    target_size=(256, 256),  # Görüntülerin yeniden boyutlandırılacağı hedef boyutlar (256x256 piksel).
    batch_size=32,  # Her bir test grubunda modele aynı anda verilecek 32 görüntülük grup.
    class_mode='categorical'  # Sınıflar arasında kategorik sınıflandırma yapılacağını belirtir.
)

Found 2522 images belonging to 2 classes.


In [5]:
# CNN modeli tanımlanıyor.
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),  # İlk evrişim katmanı, 32 filtre kullanıyor ve giriş boyutunu tanımlıyor.
    MaxPooling2D(pool_size=(2, 2)),  # Havuzlama katmanı, boyutları küçültmek için 2x2 filtre kullanıyor.

    Conv2D(64, (3, 3), activation='relu'),  # İkinci evrişim katmanı, 64 filtre kullanıyor.
    MaxPooling2D(pool_size=(2, 2)),  # İkinci havuzlama katmanı.

    Conv2D(128, (3, 3), activation='relu'),  # Üçüncü evrişim katmanı, 128 filtre kullanıyor.
    MaxPooling2D(pool_size=(2, 2)),  # Üçüncü havuzlama katmanı.

    Conv2D(256, (3, 3), activation='relu'),  # Dördüncü evrişim katmanı, 256 filtre kullanıyor.
    MaxPooling2D(pool_size=(2, 2)),  # Dördüncü havuzlama katmanı.

    Conv2D(256, (3, 3), activation='relu'),  # Beşinci evrişim katmanı, 256 filtre kullanıyor.
    MaxPooling2D(pool_size=(2, 2)),  # Beşinci havuzlama katmanı.

    Conv2D(512, (3, 3), activation='relu'),  # Altıncı evrişim katmanı, 512 filtre kullanıyor.
    MaxPooling2D(pool_size=(2, 2)),  # Altıncı havuzlama katmanı.

    Flatten(),  # 2D verileri tek boyutlu bir vektöre dönüştürüyor.
    Dense(512, activation='relu'),  # Tam bağlantılı katman, 512 nöron içeriyor.
    Dropout(0.5),  # Aşırı öğrenmeyi önlemek için %50 dropout uyguluyor.

    Dense(256, activation='relu'),  # 256 nöronlu tam bağlantılı bir başka katman.
    Dropout(0.5),  # %50 dropout uyguluyor.

    Dense(128, activation='relu'),  # 128 nöronlu tam bağlantılı katman.
    Dropout(0.5),  # %50 dropout uyguluyor.

    Dense(2, activation='softmax')  # Çıkış katmanı, 2 sınıf için softmax aktivasyonu kullanıyor.
])


In [6]:
# Modelin derlenmesi (compile) işlemi yapılıyor.
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Optimizasyon algoritması olarak Adam kullanılıyor ve öğrenme oranı 0.0001 olarak ayarlanıyor.
    loss='categorical_crossentropy',  # Çok sınıflı sınıflandırma için kullanılan kayıp fonksiyonu.
    metrics=['accuracy']  # Modelin performansını doğruluk (accuracy) metriği ile değerlendirin.
)

In [10]:
# Modelin özet bilgilerini yazdırır.
model.summary()  # Modelin katmanlarını, parametre sayısını ve çıkış şekillerini gösterir.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 128)      0

In [8]:
# Eğitim sırasında aşırı öğrenmeyi önlemek ve gereksiz eğitimden kaçınmak için erken durdurma (early stopping) callback'i tanımlanıyor.
early_stopping = EarlyStopping(monitor='loss', patience=5)  # Eğitim sırasında kayıp (loss) değerini izler, 5 epoch boyunca iyileşme olmazsa eğitimi durdurur.


In [ ]:
# Modelin eğitim süreci başlatılıyor.
model.fit(
    training_set,  # Eğitim için kullanılan veri kümesi.
    epochs=50,  # Eğitim süreci için maksimum epoch sayısı.
    batch_size=16,  # Her bir batch'te kullanılacak veri miktarı.
    callbacks=[early_stopping]  # Eğitim sırasında erken durdurma işlemini sağlayan callback.
)

In [ ]:
# Modelin test veri seti üzerindeki performansı değerlendirilir.
evaluation = model.evaluate(test_set)  # Test veri setinde loss (kayıp) ve accuracy (doğruluk) hesaplar.
print("Test Loss:", evaluation[0])  # Test setindeki kayıp değerini yazdırır.
print("Test Accuracy:", evaluation[1])  # Test setindeki doğruluk değerini yazdırır.


In [ ]:
model.save("continuum_cnn1.h5")  # Eğitilen model dosya olarak kaydediliyor.